<a href="https://colab.research.google.com/github/JinHuiXu1991/Jin_DATA606/blob/main/ipynb/DATA606_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon Product Recommender Systems
## Author: Jin Hui Xu

In [1]:
!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_product.zip?raw=true

!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_review.zip?raw=true

--2022-03-20 04:18:52--  https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_product.zip?raw=true
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/JinHuiXu1991/Jin_DATA606/raw/main/cleaned_data/cleaned_amazon_product.zip [following]
--2022-03-20 04:18:53--  https://github.com/JinHuiXu1991/Jin_DATA606/raw/main/cleaned_data/cleaned_amazon_product.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JinHuiXu1991/Jin_DATA606/main/cleaned_data/cleaned_amazon_product.zip [following]
--2022-03-20 04:18:53--  https://raw.githubusercontent.com/JinHuiXu1991/Jin_DATA606/main/cleaned_data/cleaned_amazon_product.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133,

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
import time
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


##Content Based Recommender

### Description Based Recommender

In [3]:
product_df = pd.read_csv('/content/cleaned_amazon_product.zip?raw=true', compression='zip')

In [4]:
product_df.head()

,category,description,title,brand,feature,main_cat,date,price,asin,imageURLHighRes,dateYear,dateMonth,all_text,all_text_token,word_count
0,"Appliances Refrigerators, Freezers & Ice Makers",NaN,Tupperware Freezer Square Round Container Set ...,Tupperware,Each 3-pc. set includes two 7/8-cup/200 mL and...,Appliances,2008-11-19,NaN,7301113188,[],2008.0,11.0,"Appliances Refrigerators, Freezers & Ice Maker...","['appliances', 'refrigerators', 'freezers', 'i...",51
1,"Appliances Refrigerators, Freezers & Ice Makers",2 X Tupperware Pure & Fresh Unique Covered Coo...,2 X Tupperware Pure &amp; Fresh Unique Covered...,Tupperware,2 X Tupperware Pure & Fresh Unique Covered Coo...,Appliances,2016-06-05,3.62,7861850250,['https://images-na.ssl-images-amazon.com/imag...,2016.0,6.0,"Appliances Refrigerators, Freezers & Ice Maker...","['appliances', 'refrigerators', 'freezers', 'i...",117
2,Appliances Parts & Accessories,NaN,The Cigar - Moments of Pleasure,The Cigar Book,NaN,Amazon Home,NaN,150.26,8792559360,['https://images-na.ssl-images-amazon.com/imag...,NaN,NaN,Appliances Parts & Accessories The Cigar - Mo...,"['appliances', 'parts', 'accessories', 'cigar'...",6
3,Appliances Parts & Accessories,"Multi purpost descaler, especially suited to w...",Caraselle 2X 50G Appliance Descalene,Caraselle,NaN,Tools & Home Improvement,2014-12-17,NaN,9792954481,['https://images-na.ssl-images-amazon.com/imag...,2014.0,12.0,Appliances Parts & Accessories Multi purpost d...,"['appliances', 'parts', 'accessories', 'multi'...",29
4,Appliances Parts & Accessories Range Parts & A...,Full gauge and size beveled-edge; furnished wi...,EATON Wiring 39CH-SP-L Arrow Hart 1-Gang Chrom...,EATON Wiring,Returns will not be honored on this closeout i...,Tools & Home Improvement,2007-01-16,3.43,B00002N5EL,[],2007.0,1.0,Appliances Parts & Accessories Range Parts & A...,"['appliances', 'parts', 'accessories', 'range'...",69


In [5]:
product_df.shape

(30239, 15)

In [6]:
# Replace all NaN with an empty string
product_df = product_df.fillna('')
product_df.isnull().sum()

category           0
description        0
title              0
brand              0
feature            0
main_cat           0
date               0
price              0
asin               0
imageURLHighRes    0
dateYear           0
dateMonth          0
all_text           0
all_text_token     0
word_count         0
dtype: int64

In [7]:
lem = WordNetLemmatizer()
def lemma(text):
    return ' '.join(lem.lemmatize(w) for w in text.split() if w not in stop)   

def clearn_text(doc):
  cleaned_text = re.sub('[^a-zA-Z0-9]', ' ', doc)
  cleaned_text = re.sub('\s\s+', ' ', cleaned_text)
  cleaned_text = cleaned_text.lower()
  return cleaned_text

In [8]:
product_df['description'] = product_df['description'].apply(clearn_text)
product_df['description'] = product_df['description'].apply(lemma)

In [9]:
# Define a TF-IDF Vectorizer object and remove all english stopwords
tfidf = TfidfVectorizer(stop_words='english', max_df = 0.9, min_df = 5)

In [10]:
# Construct the required TF-IDF matrix 
tfidf_matrix = tfidf.fit_transform(product_df['description'])

In [11]:
# Output the shape of tfidf_matrix
tfidf_matrix.shape

(30239, 9738)

Both linear_kernel and cosine_similarity methods produce the same result. We would like to apply both methods can see the performance of creating the cosine similarity matrix, and choose the faster one.

In [12]:
start = time.time()
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print("Time taken: %s seconds" % (time.time() - start))

Time taken: 13.096258878707886 seconds


Time taken: 14.83594274520874 seconds

In [13]:
# start = time.time()
# cosine_sim2 = cosine_similarity(tfidf_matrix, tfidf_matrix)
# print("Time taken: %s seconds" % (time.time() - start))

The performance results are very close, and the cosine_similarity result will be used

In [14]:
# Reverse mapping of indices and product id
indices = pd.Series(product_df.index, index=product_df['asin'].str.lower()).drop_duplicates()

In [15]:
# Function that takes in product id as input and gives recommendations 
def description_recommender(id, cosine_sim = cosine_sim, df = product_df, indices = indices):
  # get the index of the product that matches the id
  idx = indices[id.lower()]

  # get the pairwsie similarity scores 
  # then convert it into a list of tuples as described above
  sim_scores = list(enumerate(cosine_sim[idx]))

  # sort the product based on the cosine similarity scores
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # get the scores of the 10 most similar product. Ignore the first one because it is the input product.
  sim_scores = sim_scores[1:11]

  # get the product indices
  product_indices = [i[0] for i in sim_scores]

  # return the top 10 most similar product
  return df['asin'].iloc[product_indices].tolist(), df['title'].iloc[product_indices].tolist()

In [16]:
product_df[product_df['title'].str.contains('refrigerator', case=False)].head(1)

,category,description,title,brand,feature,main_cat,date,price,asin,imageURLHighRes,dateYear,dateMonth,all_text,all_text_token,word_count
131,"Appliances Refrigerators, Freezers & Ice Makers",compact mini cooler warmer hold 17 liter twent...,Coldmate MR-128 Mini Cooler/Warmer Deluxe Mini...,Coldmate,Press the cold button to cool to 40 F and hot ...,Appliances,2001-10-02,,B0001YH10C,['https://images-na.ssl-images-amazon.com/imag...,2001.0,10.0,"Appliances Refrigerators, Freezers & Ice Maker...","['appliances', 'refrigerators', 'freezers', 'i...",99


In [17]:
# get input id and title for the recommendation
input_id = product_df[product_df['title'].str.contains('refrigerator', case=False)].iloc[0]['asin']
input_title = product_df[product_df['title'].str.contains('refrigerator', case=False)].iloc[0]['title']

In [18]:
#Get recommendations for Coldmate MR-128 Mini Cooler/Warmer Deluxe Mini Refrigerator, input the product id 
asin, title = description_recommender(input_id)

In [19]:
print('Description Based Recommender Result for {}, {}: '.format(input_id, input_title))
for i in range(0, 10):
  print('{}. {}, {}'.format(i+1, asin[i], title[i]))

Description Based Recommender Result for B0001YH10C, Coldmate MR-128 Mini Cooler/Warmer Deluxe Mini Refrigerator: 
1. B00ID8CLMG, Avanti FF45006W 4.3 CF Frost Free Refrigerator Freezer, White
2. B00RNAH5OY, goFridge Mini Fridge Portable Electric Cooler
3. B001H80RN4, Frigidaire 241505301 Refrigerator Door Bin Genuine Original Equipment Manufacturer (OEM) Part
4. B004NEYPYQ, Frost-Free 4.3 Cu. Ft. Refrigerator/Freezer White
5. B000JLL3BK, Pek Vino Vault Wine Preserving Refrigerator, Silver
6. B01F79MKME, Amana AMA43BK Compact Single Door Refrigerator, 4.3 cu. ft, Black
7. B001F7H4RY, PORTABLE COOLER WARMER MINI FRIDGE WINE BEER
8. B001AAHW6E, Whirlpool 2179404KRA Beverage Rack
9. B001775T4C, Nostalgia Electrics CRF170RETRORED Retro Series Mini Fridge, 1.7 Cubic Feet
10. B004Y3C9J4, 1.7 cuft Superconduction Refrigerator


In [20]:
# np.save('cosine_sim', cosine_sim)

In [21]:
# original_cs = np.load("cosine_sim.npy")
# original_cs

In [22]:
# get input id and title for the recommendation
# input_id = product_df[product_df['title'].str.contains('refrigerator', case=False)].iloc[1]['asin']
# input_title = product_df[product_df['title'].str.contains('refrigerator', case=False)].iloc[1]['title']

In [23]:
#Get recommendations
# asin, title = description_recommender(input_id, cosine_sim=original_cs)

In [24]:
# print('Description Based Recommender Result for {}, {}: '.format(input_id, input_title))
# for i in range(0, 10):
#   print('{}. {}, {}'.format(i+1, asin[i], title[i]))

### Metadata based Recommender

In [25]:
product_df = pd.read_csv('/content/cleaned_amazon_product.zip?raw=true', compression='zip')
product_df = product_df.fillna('')

In [26]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30239 entries, 0 to 30238
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   category         30239 non-null  object
 1   description      30239 non-null  object
 2   title            30239 non-null  object
 3   brand            30239 non-null  object
 4   feature          30239 non-null  object
 5   main_cat         30239 non-null  object
 6   date             30239 non-null  object
 7   price            30239 non-null  object
 8   asin             30239 non-null  object
 9   imageURLHighRes  30239 non-null  object
 10  dateYear         30239 non-null  object
 11  dateMonth        30239 non-null  object
 12  all_text         30239 non-null  object
 13  all_text_token   30239 non-null  object
 14  word_count       30239 non-null  int64 
dtypes: int64(1), object(14)
memory usage: 3.5+ MB


In [27]:
# product_df['all_text'] includes product_df['category'], product_df['description'], product_df['title'], product_df['feature']

# Besides, add main_cat, price, dateYear, and dateMonth
product_df['all_text'] = product_df['all_text'] + ' ' + product_df['main_cat'] + ' ' + product_df['price'].astype(str) + ' ' + product_df['dateYear'].astype(str) + ' ' + product_df['dateMonth'].astype(str) 
product_df['all_text'] = product_df['all_text'].apply(clearn_text)
product_df['all_text'] = product_df['all_text'].apply(lemma)

In [28]:
tfidf_matrix2 = tfidf.fit_transform(product_df['all_text'])

In [29]:
tfidf_matrix2.shape

(30239, 12685)

In [30]:
cosine_sim2 = linear_kernel(tfidf_matrix2, tfidf_matrix2)

In [31]:
indices = pd.Series(product_df.index, index=product_df['asin'].str.lower()).drop_duplicates()

In [32]:
# Function that takes in product id as input and gives recommendations 
def description_recommender(id, cosine_sim = cosine_sim2, df = product_df, indices = indices):
  # get the index of the product that matches the id
  idx = indices[id.lower()]

  # get the pairwsie similarity scores 
  # then convert it into a list of tuples as described above
  sim_scores = list(enumerate(cosine_sim[idx]))

  # sort the product based on the cosine similarity scores
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # get the scores of the 10 most similar product. Ignore the first one because it is the input product.
  sim_scores = sim_scores[1:11]

  # get the product indices
  product_indices = [i[0] for i in sim_scores]

  # return the top 10 most similar product
  return df['asin'].iloc[product_indices].tolist(), df['title'].iloc[product_indices].tolist()

In [33]:
product_df[product_df['title'].str.contains('refrigerator', case=False)].head(1)

,category,description,title,brand,feature,main_cat,date,price,asin,imageURLHighRes,dateYear,dateMonth,all_text,all_text_token,word_count
131,"Appliances Refrigerators, Freezers & Ice Makers",This compact mini cooler and warmer holds 17 l...,Coldmate MR-128 Mini Cooler/Warmer Deluxe Mini...,Coldmate,Press the cold button to cool to 40 F and hot ...,Appliances,2001-10-02,,B0001YH10C,['https://images-na.ssl-images-amazon.com/imag...,2001.0,10.0,appliance refrigerator freezer ice maker compa...,"['appliances', 'refrigerators', 'freezers', 'i...",99


In [34]:
# get input id and title for the recommendation
input_id = product_df[product_df['title'].str.contains('refrigerator', case=False)].iloc[0]['asin']
input_title = product_df[product_df['title'].str.contains('refrigerator', case=False)].iloc[0]['title']

In [35]:
#Get recommendations
asin2, title2 = description_recommender(input_id, cosine_sim=cosine_sim2)

In [36]:
print('Description Based Recommender Result for {}, {}: '.format(input_id, input_title))
for i in range(0, 10):
  print('{}. {}, {}'.format(i+1, asin2[i], title2[i]))

Description Based Recommender Result for B0001YH10C, Coldmate MR-128 Mini Cooler/Warmer Deluxe Mini Refrigerator: 
1. B00RNAH5OY, goFridge Mini Fridge Portable Electric Cooler
2. B00ND5CWAA, Phoenix USB 5v Portable One Zip-top Can Cooler-mini Car Compact Refrigerator and Warmer
3. B01D3O9UB4, Knox Electric Cooler and Warmer for Car and Home with Wheels - 48 Quart (45 Liter) &ndash; Holds 60 Cans or 6 Two Liter Bottles and 15 Cans - Dual 110V AC House and 12V DC Vehicle Plugs
4. B001F7H4RY, PORTABLE COOLER WARMER MINI FRIDGE WINE BEER
5. B00TY2KP9G, Magic Chef MCBR440B2 Refrigerator, 4.4 cu. ft, Black
6. B001775T4C, Nostalgia Electrics CRF170RETRORED Retro Series Mini Fridge, 1.7 Cubic Feet
7. B00YNNEC8Q, Mini Wine Cooler
8. B00OY8EILG, Edgestar 84 Can Extreme Cool Beverage Cooler &amp; 32 Bottle Dual Zone Stainless Steel Wine Cooler
9. B00ID8CLMG, Avanti FF45006W 4.3 CF Frost Free Refrigerator Freezer, White
10. B0187KYRQC, Coca-Cola Mini Can Cooler


In [37]:
print('Description Based Recommender Result for {}, {}: '.format(input_id, input_title))
for i in range(0, 10):
  print('{}. {}, {}'.format(i+1, asin[i], title[i]))

Description Based Recommender Result for B0001YH10C, Coldmate MR-128 Mini Cooler/Warmer Deluxe Mini Refrigerator: 
1. B00ID8CLMG, Avanti FF45006W 4.3 CF Frost Free Refrigerator Freezer, White
2. B00RNAH5OY, goFridge Mini Fridge Portable Electric Cooler
3. B001H80RN4, Frigidaire 241505301 Refrigerator Door Bin Genuine Original Equipment Manufacturer (OEM) Part
4. B004NEYPYQ, Frost-Free 4.3 Cu. Ft. Refrigerator/Freezer White
5. B000JLL3BK, Pek Vino Vault Wine Preserving Refrigerator, Silver
6. B01F79MKME, Amana AMA43BK Compact Single Door Refrigerator, 4.3 cu. ft, Black
7. B001F7H4RY, PORTABLE COOLER WARMER MINI FRIDGE WINE BEER
8. B001AAHW6E, Whirlpool 2179404KRA Beverage Rack
9. B001775T4C, Nostalgia Electrics CRF170RETRORED Retro Series Mini Fridge, 1.7 Cubic Feet
10. B004Y3C9J4, 1.7 cuft Superconduction Refrigerator
